# Chapter 9 練習と総合問題解答

## 9.2 高度なSQL処理

### 9.2.2 exist文、case文、自己結合の応用

In [1]:
show databases

,Database
0,TEST1
1,TEST2
2,employees
3,information_schema
4,mysql
5,performance_schema
6,world


In [2]:
use TEST1

Query OK, 0 rows affected.

#### <練習問題 1>

case文を使って部署毎にgender別の人数を集計してみましょう。(特に部署移動等は考慮しなくて良いです。)

ヒント：case文をsum関数の中に記述できます。

In [3]:
use employees;
show tables;

,Tables_in_employees
0,departments
1,dept_emp
2,dept_manager
3,employees
4,salaries
5,titles


In [4]:
/*解答*/
select 
	t2.dept_no
	,sum(case when t.gender = 'M' then 1 else 0 end) MaleCount
	,sum(case when t.gender = 'F' then 1 else 0 end) FemaleCount
from employees.employees t
left outer join employees.dept_emp t2
on t2.emp_no = t.emp_no
group by t2.dept_no

,dept_no,MaleCount,FemaleCount
0,d001,12174,8037
1,d002,10331,7015
2,d003,10711,7075
3,d004,43936,29549
4,d005,51449,34258
5,d006,12039,8078
6,d007,31391,20854
7,d008,12687,8439
8,d009,14132,9448


#### <練習問題 2>

job titleの全ての組み合わせ（重複なし）を表示してみましょう。

In [5]:
/*解答*/
select a.title,b.title 
from 
(select title from titles group by title) a,
(select title from titles group by title) b
where a.title > b.title

,title,title
0,Engineer,Assistant Engineer
1,Manager,Assistant Engineer
2,Senior Engineer,Assistant Engineer
3,Senior Staff,Assistant Engineer
4,Staff,Assistant Engineer
5,Technique Leader,Assistant Engineer
6,Manager,Engineer
7,Senior Engineer,Engineer
8,Senior Staff,Engineer
9,Staff,Engineer


21通りになっているのがわかります。

#### <練習問題 3>

exists文を使って部署毎に現在job titleがStaffである人数を集計してみましょう。なお、現在とは、to_dateが9999-01-01のことを指すとします。

In [6]:
/*解答1*/
select dept_no,count(*) as staff_num from
(
select * from (select * from dept_emp where to_date = '9999-01-01') c
where exists ( select * from 
(select a.emp_no from titles a where title = 'staff' and a.to_date = '9999-01-01') b
where c.emp_no = b.emp_no )
) as staff_dept
group by dept_no;

,dept_no,staff_num
0,d001,3551
1,d002,2891
2,d003,3073
3,d004,349
4,d005,315
5,d007,8903
6,d008,2870
7,d009,3574


In [7]:
/*別解*/
select t2.dept_no,count(t.emp_no)
from (select * from employees.titles 
where to_date = '9999-01-01' and title ='Staff'
) t
left outer join 
(select * from employees.dept_emp 
where to_date = '9999-01-01') t2
on t.emp_no = t2.emp_no
group by t2.dept_no;

,dept_no,count(t.emp_no)
0,d001,3551
1,d002,2891
2,d003,3073
3,d004,349
4,d005,315
5,d007,8903
6,d008,2870
7,d009,3574


### 9.2.3 外部結合の応用

#### <練習問題 1>

各従業員について、最大給料が70,000以下の人、70,000～80,000の人、80,000～90,000の人、90,000より多い人でクロス表を作成してみましょう。

In [8]:
/*解答*/
select t.emp_no,max(t.salary),
case when max(t.salary) <= 70000 then 'Yes' else '' end as `under70,000`,
case when max(t.salary) >  70000 and max(t.salary) <= 80000 then 'Yes' else '' end as `from_70,000_to_80,000`,
case when max(t.salary) >  80000 and max(t.salary) <= 90000 then 'Yes' else '' end as `from_80,000_to_90,000`,
case when max(t.salary) >  90000 then 'Yes' else '' end as `over90,000`
from employees.salaries t
group by t.emp_no
limit 10;

,emp_no,max(t.salary),"under70,000","from_70,000_to_80,000","from_80,000_to_90,000","over90,000"
0,10001,88958,,,Yes,
1,10002,72527,,Yes,,
2,10003,43699,Yes,,,
3,10004,74057,,Yes,,
4,10005,94692,,,,Yes
5,10006,60098,Yes,,,
6,10007,88070,,,Yes,
7,10008,52668,Yes,,,
8,10009,94443,,,,Yes
9,10010,80324,,,Yes,


#### <練習問題 2>

最大の給料が100,000より多い人のうち、最小の給料が90,000以上の人の数を求めてみましょう。

In [9]:
/*解答*/
select count(a.emp_no)
from 
(select emp_no from employees.salaries t group by emp_no having max(salary) > 100000) a
left outer join 
(select emp_no from employees.salaries t group by emp_no having min(salary) >= 90000) b
on a.emp_no = b.emp_no
where b.emp_no is not null

,count(a.emp_no)
0,4835


## 9.3 ストアドプロシージャとスドアドファンクション
ゴール：ストアドプロシージャについて操作できること

### 9.3.1 ストアドプロシージャ

#### <練習問題 1>

次の処理を実行するストアド・プロシージャを作成して実行してみましょう。

・meiboテーブルと同じ構造のテーブルmeibo3を作成

・meiboテーブルのデータをmeibo3テーブルに挿入

・meibo3テーブルにweightカラムを追加

・引数で与えられたBMIとheightカラムの身長から体重を逆算してweightカラムに設定する（※BMI=体重(kg)÷(身長(m)×身長(m))

・meibo3の内容を出力

In [10]:
/*解答*/
/*データベース選択*/
use TEST1;
/*これから作成するストアドプロシージャと同じ物があれば削除*/
drop procedure if exists practice1;

Query OK, 0 rows affected. Query OK, 0 rows affected.

In [11]:
create procedure TEST1.practice1(in bmi int)
begin
/*これから作成するmeibo3テーブルが既にアレば削除*/
drop table if exists meibo3; 
/* meiboテーブルからテーブル構造をコピー*/
create table TEST1.meibo3 like TEST1.meibo;
/* meiboテーブルからデータをコピー*/
insert into  meibo3 select * from meibo;
/* weightカラムを作成*/
alter table TEST1.meibo3 add weight int null; 
/*引数で与えたBMIから体重を逆算*/
update TEST1.meibo3 set weight = pow(height/100,2) * bmi;
/*meibo3の内容を出力*/
select * from meibo3;
end

Query OK, 0 rows affected.

In [12]:
call practice1(22);

,id,name,class,age,height,weight
0,1,Yamada,1,14,150,50
1,2,Tanaka,2,13,150,50
2,3,Suzuki,1,13,150,50
3,4,Kato,2,15,150,50
4,5,Ito,3,12,155,53
5,6,Takeuchi,2,16,155,53
6,7,Kimura,3,11,160,56
7,8,Sato,1,14,160,56
8,100,dummy_Yamada,None,None,None,None
9,200,dummy_Tanaka,None,None,None,None


### 9.3.2 ストアドファンクション

#### <練習問題 1>

上記の例を参考に、meiboテーブルから平均身長を返す関数を作成して呼んでください。

In [13]:
use TEST1;
drop function if exists fc2;

Query OK, 0 rows affected. Query OK, 0 rows affected.

In [14]:
create function fc2() returns double
begin
declare avgHeight double;
select avg(height) into avgHeight from meibo;
return avgHeight;
end

Query OK, 0 rows affected.

In [15]:
select fc2();

,fc2()
0,153.75


#### <練習問題 2>

MySQLはナノ秒の取扱ができません。時、分、秒、ナノ秒（小数点以下の秒）の4つの引数を与えると "HH:MM:SS.sssssssss"形式の文字列(varchar(18))を返す関数を作って実行してみましょう。

ヒント：文字列の結合にはconcat関数が利用できます。

In [16]:
use TEST1;
drop function if exists getTime;

Query OK, 0 rows affected. Query OK, 0 rows affected.

In [17]:
create function getTime(hour int,min int,sec int,nano_sec varchar(9)) returns varchar(18)
begin
    return concat(hour,":",min,":",sec,".",nano_sec);
end

Query OK, 0 rows affected.

In [18]:
select getTime(10,35,25,123);

,"getTime(10,35,25,123)"
0,10:35:25.123


In [19]:
select getTime(10,35,25,'012345678');

,"getTime(10,35,25,'012345678')"
0,10:35:25.012345678


***

## 9.4 インデクシングとSQLの高速化

### 9.4.1 インデックスの設定
キーワード：インデックス

#### <練習問題 1>

dept_empテーブルのfrom_date、to_dateに複合インデックスを設定してみましょう。

In [20]:
/*データベースの選択とインデックスの確認をします*/
use employees;
show index from dept_emp;

,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment
0,dept_emp,0,PRIMARY,1,emp_no,A,331570,None,None,,BTREE,,
1,dept_emp,0,PRIMARY,2,dept_no,A,331570,None,None,,BTREE,,
2,dept_emp,1,emp_no,1,emp_no,A,331570,None,None,,BTREE,,
3,dept_emp,1,dept_no,1,dept_no,A,331570,None,None,,BTREE,,


In [21]:
/*解答*/
create index from_to_date on dept_emp(from_date, to_date);

Query OK, 0 rows affected.

In [22]:
/*確認*/
show index from dept_emp;

,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment
0,dept_emp,0,PRIMARY,1,emp_no,A,331570,None,None,,BTREE,,
1,dept_emp,0,PRIMARY,2,dept_no,A,331570,None,None,,BTREE,,
2,dept_emp,1,emp_no,1,emp_no,A,331570,None,None,,BTREE,,
3,dept_emp,1,dept_no,1,dept_no,A,16,None,None,,BTREE,,
4,dept_emp,1,from_to_date,1,from_date,A,13815,None,None,,BTREE,,
5,dept_emp,1,from_to_date,2,to_date,A,331570,None,None,,BTREE,,


## 9.6 総合問題

### 9.6.1 総合問題1

以下のデータベースを使って、次の問題を考えて実行してみましょう。

https://dev.mysql.com/doc/sakila/en/

(1) 支払金額の総額が100ドル以上の顧客名と支払総額を表示してください。

(2) (1)の結果を新しいテーブルに格納するストアド・プロシージャを作成してください。ただし、支払総額の閾値は引数で指定できるようにしてください。

In [23]:
/*解答(1)*/
select 	concat(c.first_name," ",c.last_name)  customerName,
		sum(p.amount) sumAmount
from sakila.payment p
left outer join sakila.customer c
on p.customer_id = c.customer_id
group by p.customer_id
having sum(p.amount) > 100
order by sumAmount desc


,customerName,sumAmount
0,KARL SEAL,221.55
1,ELEANOR HUNT,216.54
2,CLARA SHAW,195.58
3,MARION SNYDER,194.61
4,RHONDA KENNEDY,194.61
5,TOMMY COLLAZO,186.62
6,WESLEY BULL,177.60
7,TIM CARY,175.61
8,MARCIA DEAN,175.58
9,ANA BRADLEY,174.66


In [24]:
/*解答(2)*/
use sakila; 
drop procedure if exists practice; 

Query OK, 0 rows affected. Query OK, 0 rows affected.

In [25]:
create procedure sakila.practice(in threshold DECIMAL(20,2))
begin
drop table if exists newTable;
create table sakila.newTable (
	`customerName` varchar(91) NOT NULL,
	`sumPaymentAmount`  DECIMAL(20,2) NOT NULL,
	`last_update` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
	PRIMARY KEY (`customerName`)
);

insert into sakila.newTable 
select 	concat(c.first_name," ",c.last_name)  customerName,
			sum(p.amount) sumAmount,
			null
from sakila.payment p
left outer join sakila.customer c
on p.customer_id = c.customer_id
group by p.customer_id
having sum(p.amount) > threshold;

end

Query OK, 0 rows affected.

In [26]:
call practice(120);
select * from sakila.newTable order by sumPaymentAmount desc limit 100;

,customerName,sumPaymentAmount,last_update
0,KARL SEAL,221.55,2017-12-20 10:57:15
1,ELEANOR HUNT,216.54,2017-12-20 10:57:15
2,CLARA SHAW,195.58,2017-12-20 10:57:15
3,RHONDA KENNEDY,194.61,2017-12-20 10:57:15
4,MARION SNYDER,194.61,2017-12-20 10:57:15
5,TOMMY COLLAZO,186.62,2017-12-20 10:57:15
6,WESLEY BULL,177.60,2017-12-20 10:57:15
7,TIM CARY,175.61,2017-12-20 10:57:15
8,MARCIA DEAN,175.58,2017-12-20 10:57:15
9,ANA BRADLEY,174.66,2017-12-20 10:57:15


### 9.2.2 総合問題2

以下のデータベースを使って 後の問に答えてください。

https://sites.google.com/a/dlpage.phi-integration.com/pentaho/mondrian/mysql-foodmart-database/foodmart_mysql.tar.gz?attredirects=0

解説はこちら

https://github.com/julianhyde/foodmart-data-hsqldb

http://www.fromdual.ch/foodmart-old

(1) 1998年1月の従業員の給料ランキングを10位まで表示してください。

(2) sales_fact_1998テーブルの中で、同じ人が同じ時間に買っている商品の2つの組み合わせの中で、購買回数が多い2つの商品はどれとどれでしょうか。TOP5を抽出して、商品名(product_name)でそれぞれ答えてください。なお、商品はproduct_idベースでカウントして、同じ人とはcustomer_idが同じことを意味し、同じ時間とはtime_idが同じことを意味します。なお、これをバスケット分析（同時併売）といい、期間併売で計算することもあります。

・準備

In [27]:
drop database if exists foodmart;

Query OK, 0 rows affected.

In [28]:
create database foodmart;

Query OK, 1 rows affected.

上記のようにデータベースを用意したあとは、以下を実行。

-------------------
mkdir foodmart  
cd foodmart  
wget http://pentaho.dlpage.phi-integration.com/mondrian/mysql-foodmart-database/foodmart_mysql.tar.gz  
tar zxf  foodmart_mysql.tar.gz  
mysql -h"db1" -P"3306" -u root -p"ilect" foodmart < foodmart_mysql.sql  

-------------------

In [29]:
use foodmart;

Query OK, 0 rows affected.

In [30]:
show tables;

,Tables_in_foodmart
0,account
1,agg_c_10_sales_fact_1997
2,agg_c_14_sales_fact_1997
3,agg_c_special_sales_fact_1997
4,agg_g_ms_pcat_sales_fact_1997
5,agg_l_03_sales_fact_1997
6,agg_l_04_sales_fact_1997
7,agg_l_05_sales_fact_1997
8,agg_lc_06_sales_fact_1997
9,agg_lc_100_sales_fact_1997


解答　(1)

In [31]:
/*解答(1)*/
select
	main.employee_id,
	main.salary,
	(
		select count(b.pay_date)
		from foodmart.salary b
		where main.salary < b.salary_paid and DATE_FORMAT(b.pay_date,'%Y%m') = '199801'
	) + 1 as rank
from
(
select
	a.employee_id,
	sum(a.salary_paid) salary
from foodmart.salary a
where DATE_FORMAT(a.pay_date,'%Y%m') = '199801'
group by a.employee_id
order by 2 desc
) main
order by main.salary desc
limit 10;

,employee_id,salary,rank
0,14,154.5455,1
1,16,154.5455,1
2,9,154.5455,1
3,13,136.3636,4
4,11,136.3636,4
5,946,118.1818,6
6,1066,118.1818,6
7,821,118.1818,6
8,885,118.1818,6
9,748,118.1818,6


解答　(2)

In [32]:
/*解答(2)*/
select d.product_name as prod_name_1,e.product_name  as prod_name_2,c.cnt
from (select a.product_id as prod_1,b.product_id as prod_2,count(*) as cnt
from sales_fact_1998 a,sales_fact_1998 b 
where a.product_id > b.product_id and a.time_id=b.time_id and a.customer_id=b.customer_id 
group by 1,2 order by 3 desc limit 5) c 
left outer join product d on c.prod_1 = d.product_id 
left outer join product e on c.prod_2 = e.product_id ;

,prod_name_1,prod_name_2,cnt
0,Big Time Apple Cinnamon Waffles,Carlson Low Fat Sour Cream,12
1,Great Wheat Bread,Best Choice Salsa Dip,11
2,Plato Strawberry Jelly,Tell Tale New Potatos,11
3,Big Time Frozen Chicken Breast,Better Fancy Canned Clams,10
4,Tell Tale Tomatos,Denny 60 Watt Lightbulb,10
